In [ ]:
!pip install -q langchain_openai==0.0.2 faiss-cpu==1.7.4 openai==1.6.1 tiktoken==0.5.2 langchain_community==0.0.11 langchain==0.1.0

In [ ]:
# По запросу вводим ключ OPENAI_API_KEY и не забываем нажать ENTER.

import getpass
import os
import openai
from openai import OpenAI
openai_key = getpass.getpass("OpenAI API Key:")
os.environ["OPENAI_API_KEY"] = openai_key
openai.api_key = openai_key
# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI()

# Загрузка БЗ

In [ ]:
import re
import requests
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from datetime import datetime


In [ ]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [ ]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/...')

In [ ]:
data_from_url

In [ ]:
source_chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={"meta":"data"}))
len(source_chunks)

In [ ]:
source_chunks[0]

In [ ]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

# ConversationBufferMemory

ConversationBufferMemory сохраняет весь разговор в буферной памяти до разрешенного максимального предела (например, 4096 для gpt-3.5-turbo, 8192 для gpt-4). 

In [ ]:
# создаем функцию, которая будет получать ответ от модели на основании промптов, чанков и истории беседы с пользователем:
def answer_user_quest(system, topic, instructions, search_index, temp, verbose, k, model, hist=''):
    docs = db.similarity_search_with_score(topic, k=k)
    message_content = '\n '.join([f'Отрывок текста №{i+1}\n{doc[0].page_content}' for i, doc in enumerate(docs)])
    messages = [{"role": "system", "content": system}, {"role": "user", "content": f"{instructions}\n\nТексты для анализа:\n{message_content}\n\nИстория чата:\n{hist}\n\nВопрос клиента: {topic}"}]

    completion = client.chat.completions.create(model=model, messages=messages, temperature=temp)
    return message_content, completion.choices[0].message.content

In [ ]:
# определяем переменные и промпты
system_prompt = """Ты-консультант, отвечающий на вопрос пользователя на основе документа с информацией. Не придумывай ничего от себя, отвечай максимально по документу. Говори только на русском языке.
"""
model = "gpt-3.5-turbo-1106"
temperature = 0.1
num_fragment = 3
verbose = 0
user_prompt = "Если пользователь с тобой поздоровался - ответь на его приветствие. Затем ответь на вопрос пользователя на основании представленных тебе текстов для анализа, а также принимая во внимание историю чата."

In [ ]:
# Создаем экземпляр класса для работы с памятью диалогов
memory = ConversationBufferMemory(return_messages=True)

# Переменная для хранения вопроса пользователя
user_question = ''

while True:
  # Получаем вопрос от клиента
  user_question = input('\nВопрос клиента: ')

  # Проверяем, не является ли команда ввода командой остановки
  if user_question.lower() in ['stop', 'стоп']:
      break

  # Загружаем историю сообщений из памяти
  history = memory.load_memory_variables({})

  # Формируем строку истории разговора из загруженной истории
  conversation_string_from_history = "\n".join(message.content for message in history['history'])


  # модель должна ответить на вопрос в соответствии с промптом:
  chunks, output = answer_user_quest(system_prompt, user_question, user_prompt, db, temperature, verbose, num_fragment, model, hist=conversation_string_from_history)

  # Сохраняем вопрос пользователя и ответ модели в историю
  memory.save_context({"input": user_question}, {"output": output})

  # Выводим ответ модели
  print("Ответ:\n", output)
  # Выводим текущую строку истории разговора
  print("conversation_string_from_history: ", conversation_string_from_history)



Вопрос клиента: Привет! Как дела?
Ответ:
 Привет! Я рад помочь вам. Чем могу быть полезен?
conversation_string_from_history:  

Вопрос клиента: Кто женился на Хатсуне Мику?
Ответ:
 Привет! Хатсуне Мику - это голограмма, которая стала популярной певицей. Ее "мужем" является Акихико Кондо, который провел свадьбу с ней в Токио и даже хранит куклу Мику в своей спальне.
conversation_string_from_history:  Привет! Как дела?
Привет! Я рад помочь вам. Чем могу быть полезен?

Вопрос клиента: Какте угрозы представляет ИИ для общества?
Ответ:
 Привет! Искусственный интеллект представляет различные угрозы для общества, как указано в представленных текстах. В первом отрывке говорится о возможности использования систем искусственного интеллекта для убеждения людей в определенном поведении, включая голосование. Также упоминается возможность влияния частных компаний на социальные сети, что создает новую ситуацию и угрозу для общества. Во втором отрывке приводится мнение о том, что искусственный интелл

# ConversationSummaryMemory

ConversationSummaryMemory суммаризует историю разговора перед передачей её в параметр history. Таким образом, можно сократить объем текста, обрабатываемого моделью, и улучшить скорость обработки запросов без существенной потери качества.

In [ ]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI

In [ ]:
# Создаем экземпляр класса для работы с памятью диалогов
summary_memory = ConversationSummaryMemory(llm=ChatOpenAI(model_name=model))

# Переменная для хранения вопроса пользователя
user_question = ''

while True:
  # Получаем вопрос от клиента
  user_question = input('\nВопрос клиента: ')

  # Проверяем, не является ли команда ввода командой остановки
  if user_question.lower() in ['stop', 'стоп']:
      break

  # Загружаем историю сообщений из памяти
  history = summary_memory.load_memory_variables({})

  # Проверяем, является ли история списком сообщений или строкой
  if isinstance(history['history'], str):
      conversation_string_from_history = history['history']
  else:
      # Если история - это список сообщений, то формируем строку из их содержимого
      conversation_string_from_history = "\n".join(message.content for message in history['history'])

  # модель должна ответить на вопрос в соответствии с промптом:
  chunks, output = answer_user_quest(system_prompt, user_question, user_prompt, db, temperature, verbose, num_fragment, model, hist=conversation_string_from_history)

  # Сохраняем вопрос пользователя и ответ модели в историю
  summary_memory.save_context({"input": user_question}, {"output": output})

  # Выводим ответ модели
  print("Ответ:\n", output)
  # Выводим текущую строку истории разговора
  print("conversation_string_from_history: ", conversation_string_from_history)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(



Вопрос клиента: Привет как дела?
Ответ:
 Привет! Как дела?
conversation_string_from_history:  

Вопрос клиента: ИИ должен контролироваться правительством?
Ответ:
 Привет! Как дела? Относительно вашего вопроса, в текстах для анализа упоминается угроза, которую представляют системы искусственного интеллекта для процесса принятия политических решений. Также обсуждается возможность влияния частных компаний на системы искусственного интеллекта, которые могут использоваться для убеждения людей вести себя определенным образом. В контексте автономного оружия также поднимается вопрос этических проблем. Таким образом, вопрос контроля над системами искусственного интеллекта, включая автономное оружие, вызывает серьезное разногласие и требует внимательного изучения.
conversation_string_from_history:  The human greets the AI in Russian and asks how it is doing. The AI responds in Russian and asks how the human is doing.

Вопрос клиента: Как ИИ может помогать людям с психологическими расстройствами

# ConversationBufferWindow Memory

ConversationKGMemory сохраняет ключевые факты и предпочтения из диалога в виде структурированных данных (графа знаний), чтобы учитывать контекст в последующих беседах. Эта информация используется для персонализации ответов, её можно редактировать или удалять по запросу.

ConversationBufferWindow Memory добавляет окно в буферную память, запоминая только последние k итераций диалога. Это помогает сократить количество используемых токенов, но, следовательно, теряет контекст любых входных данных, предшествующих предыдущим K взаимодействиям.

In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI

In [ ]:
# Создаем экземпляр класса для работы с памятью диалогов
# в данном случае указываем параметр k = 3, что означает, что будут сохраняться последние 3 пары вопрос-ответ
# по умолчанию k = 5
window_memory = ConversationBufferWindowMemory(llm=ChatOpenAI(model_name=model), k= 3)

# Переменная для хранения вопроса пользователя
user_question = ''

while True:
  # Получаем вопрос от клиента
  user_question = input('\nВопрос клиента: ')

  # Проверяем, не является ли команда ввода командой остановки
  if user_question.lower() in ['stop', 'стоп']:
      break

  # Загружаем историю сообщений из памяти
  history = window_memory.load_memory_variables({})

  # Проверяем, является ли история списком сообщений или строкой
  if isinstance(history['history'], str):
      conversation_string_from_history = history['history']
  else:
      # Если история - это список сообщений, то формируем строку из их содержимого
      conversation_string_from_history = "\n".join(message.content for message in history['history'])

  # модель должна ответить на вопрос в соответствии с промптом:
  chunks, output = answer_user_quest(system_prompt, user_question, user_prompt, db, temperature, verbose, num_fragment, model, hist=conversation_string_from_history)

  # Сохраняем вопрос пользователя и ответ модели в историю
  window_memory.save_context({"input": user_question}, {"output": output})

  # Выводим ответ модели
  print("Ответ:\n", output)
  # Выводим текущую строку истории разговора
  print("conversation_string_from_history: ", conversation_string_from_history)



Вопрос клиента: Привет! Меня зовут Настя
Ответ:
 Привет, Настя! Какой вопрос у тебя есть?
conversation_string_from_history:  

Вопрос клиента: Как ИИ может помочь людям?
Ответ:
 Искусственный интеллект (ИИ) может помочь людям в различных областях, таких как медицина, производство, транспорт, образование и многие другие. Например, в медицине ИИ может использоваться для анализа медицинских изображений, диагностики заболеваний и разработки индивидуальных лечебных рекомендаций. В производстве ИИ может оптимизировать процессы и повышать эффективность производства. В транспорте ИИ может улучшать безопасность и управление транспортными средствами. Также ИИ может помогать в образовании, предоставляя персонализированные образовательные материалы и помогая студентам в обучении.
conversation_string_from_history:  Human: Привет! Меня зовут Настя
AI: Привет, Настя! Какой вопрос у тебя есть?

Вопрос клиента: Расскажи анекдот про ИИ
Ответ:
 К сожалению, у меня нет анекдотов про искусственный интелле

# ConversationSummaryBufferMemory

Память ConversationSummaryBuffer представляет собой комбинацию памяти ConversationSummary и памяти ConversationBufferWindow. Как следует из названия, он сохраняет сводку предыдущих взаимодействий в формате саммари, а также окно предыдущих взаимодействий в их необработанном формате. таким образом, недавние разговоры доступны с полным контекстом, при этом не теряется важная информация в прошлом в виде более короткого резюме. Это снижает общее использование токена.

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# Создаем экземпляр класса для работы с памятью диалогов
# max_token_limit: Максимальное количество токенов, которое может храниться в буфере. По умолчанию установлено значение 2000.
SummaryBuffer_memory = ConversationSummaryBufferMemory(llm=ChatOpenAI(model_name=model), max_token_limit = 300)

# Переменная для хранения вопроса пользователя
user_question = ''

while True:
  # Получаем вопрос от клиента
  user_question = input('\nВопрос клиента: ')

  # Проверяем, не является ли команда ввода командой остановки
  if user_question.lower() in ['stop', 'стоп']:
      break

  # Загружаем историю сообщений из памяти
  history = SummaryBuffer_memory.load_memory_variables({})

  # Проверяем, является ли история списком сообщений или строкой
  if isinstance(history['history'], str):
      conversation_string_from_history = history['history']
  else:
      # Если история - это список сообщений, то формируем строку из их содержимого
      conversation_string_from_history = "\n".join(message.content for message in history['history'])

  # модель должна ответить на вопрос в соответствии с промптом:
  chunks, output = answer_user_quest(system_prompt, user_question, user_prompt, db, temperature, verbose, num_fragment, model, hist=conversation_string_from_history)

  # Сохраняем вопрос пользователя и ответ модели в историю
  SummaryBuffer_memory.save_context({"input": user_question}, {"output": output})

  # Выводим ответ модели
  print("Ответ:\n", output)
  # Выводим текущую строку истории разговора
  print("conversation_string_from_history: ", conversation_string_from_history)



Вопрос клиента: Привет. Я Настя
Ответ:
 Привет, Настя! Как я могу помочь тебе сегодня?
conversation_string_from_history:  

Вопрос клиента: Напиши три интересных факта про ИИ
Ответ:
 Привет, Настя! Вот три интересных факта про искусственный интеллект:

1. Искусственный интеллект становится все более актуальным и важным в нашей жизни, и в будущем мы будем все больше взаимодействовать с технологиями, использующими ИИ.

2. Искусственный интеллект может повлиять на политический процесс и демократию, предлагая определенные преимущества, но также существует опасность создания глобальной диктатуры.

3. Некоторые эксперты считают, что ИИ может изменить политику к лучшему, поскольку представляет собой чистый разум, но также существует опасность игнорирования интересов общества и преследования собственных целей.

Если у тебя есть еще вопросы, не стесняйся спрашивать!
conversation_string_from_history:  Human: Привет. Я Настя
AI: Привет, Настя! Как я могу помочь тебе сегодня?

Вопрос клиента: Как

# ConversationKGMemory

In [ ]:
from langchain.memory import ConversationKGMemory

# Создаем экземпляр класса для работы с памятью диалогов
convKGMemory = ConversationKGMemory(llm=ChatOpenAI(model_name=model))

# Переменная для хранения вопроса пользователя
user_question = ''


In [ ]:
while True:
    # Получаем вопрос от клиента
    user_question = input('\nВопрос клиента: ')

    # Проверяем, не является ли команда ввода командой остановки
    if user_question.lower() in ['stop', 'стоп']:
        break

    # Обновляем контекст
    convKGMemory.save_context({"input": user_question}, {"output": ""})

    # Загружаем историю сообщений из памяти
    history = convKGMemory.load_memory_variables({"input": user_question})
    history_content = history[convKGMemory.memory_key]  # Используем memory_key для доступа к истории

    # Формируем строку истории из содержимого
    if isinstance(history_content, list):
        conversation_string_from_history = "\n".join(message.content for message in history_content)
    else:
        conversation_string_from_history = history_content

    chunks, output = answer_user_quest(system_prompt, user_question, user_prompt, db, temperature, verbose, num_fragment, model, hist=conversation_string_from_history)

    # Сохраняем вопрос пользователя и ответ модели в историю
    convKGMemory.save_context({"input": user_question}, {"output": output})

    # Выводим ответ модели
    print("Ответ:\n", output)
    # Выводим текущую строку истории разговора
    print("conversation_string_from_history: ", conversation_string_from_history)



Вопрос клиента: Привет! 
Ответ:
 Привет! Какой вопрос у тебя есть?
conversation_string_from_history:  

Вопрос клиента: ИИ может уничтожить человечество?
Ответ:
 Привет! Касательно вашего вопроса, в текстах, которые вы предоставили, говорится о потенциальной опасности развития искусственного интеллекта в области военных технологий. Однако, в тексте также упоминается, что разработчики искусственного интеллекта стремятся использовать его для улучшения мира. Таким образом, возможность уничтожения человечества обсуждается в контексте этических и политических вопросов, но не является единственной перспективой развития искусственного интеллекта.
conversation_string_from_history:  

Вопрос клиента: Выдели несколько самых странных фактов об ИИ
Ответ:
 Привет! Вот несколько интересных фактов об искусственном интеллекте:

1. В Токио старший советник группы Soft Bank и владелец собственной консалтинговой компании считает, что ИИ может изменить политику к лучшему, поскольку искусственный интеллек